# Sélection de variables : une introduction

Lino Galiana  
2026-01-11

<div class="badge-container"><div class="badge-text">Pour essayer les exemples présents dans ce tutoriel :</div><a href="https://github.com/linogaliana/python-datascientist-notebooks/blob/main/notebooks/modelisation/4_featureselection.ipynb" target="_blank" rel="noopener"><img src="https://img.shields.io/static/v1?logo=github&label=&message=View%20on%20GitHub&color=181717" alt="View on GitHub"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/vscode-python?autoLaunch=true&name=«4_featureselection»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-vscode.sh»&init.personalInitArgs=«modelisation%204_featureselection»" target="_blank" rel="noopener"><img src="https://custom-icon-badges.demolab.com/badge/SSP%20Cloud-Lancer_avec_VSCode-blue?logo=vsc&logoColor=white" alt="Onyxia"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/jupyter-python?autoLaunch=true&name=«4_featureselection»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-jupyter.sh»&init.personalInitArgs=«modelisation%204_featureselection»" target="_blank" rel="noopener"><img src="https://img.shields.io/badge/SSP%20Cloud-Lancer_avec_Jupyter-orange?logo=Jupyter&logoColor=orange" alt="Onyxia"></a>
<a href="https://colab.research.google.com/github/linogaliana/python-datascientist-notebooks-colab//blob/main//notebooks/modelisation/4_featureselection.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a><br></div>

L’ensemble de la partie *machine learning* utilise le même jeu de données, présenté dans l’[introduction de cette partie](index.qmd) : les données de vote aux élections présidentielles américaines croisées à des variables sociodémographiques. Le code est disponible [sur Github](https://github.com/linogaliana/python-datascientist/blob/main/content/modelisation/get_data.py).


In [ ]:
!pip install geopandas openpyxl plotnine plotly

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/linogaliana/python-datascientist/main/content/modelisation/get_data.py'
r = requests.get(url, allow_redirects=True)
open('getdata.py', 'wb').write(r.content)

import getdata
votes = getdata.create_votes_dataframes()

Jusqu’à présent, nous avons supposé que les variables utiles à la prévision du
vote Républicain étaient connues du modélisateur. Nous n’avons ainsi exploité qu’une partie
limitée des variables disponibles dans nos données. Néanmoins, outre le fléau
computationnel que représenterait la construction d’un modèle avec un grand
nombre de variables, le choix d’un nombre restreint de variables
(modèle parcimonieux) limite le risque de sur-apprentissage.

Comment, dès lors, choisir le bon nombre de variables et la meilleure
combinaison de ces variables ? Il existe de multiples méthodes, parmi lesquelles :

-   se fonder sur des critères statistiques de performance qui pénalisent les
    modèles non parcimonieux. Par exemple, le BIC.
-   techniques de *backward elimination*.
-   construire des modèles pour lesquels la statistique d’intérêt pénalise l’absence
    de parcimonie (ce que l’on va souhaiter faire ici).

Dans ce chapitre, nous allons présenter
les enjeux principaux de la sélection
de variables par le biais du LASSO.

Nous allons utiliser par la suite les fonctions ou
packages suivants :


In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
import sklearn.metrics
from sklearn.linear_model import LinearRegression
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.linear_model import lasso_path
import seaborn as sns

# 1. Principe du LASSO

## 1.1 Principe général

La classe des modèles de *feature selection* est ainsi très vaste et regroupe
un ensemble très divers de modèles. Nous allons nous focaliser sur le LASSO
(*Least Absolute Shrinkage and Selection Operator*)
qui est une extension de la régression linéaire qui vise à sélectionner des
modèles *sparses*. Ce type de modèle est central dans le champ du
*Compressed sensing* (où on emploie plutôt le terme
de *L1-regularization* que de LASSO). Le LASSO est un cas particulier des
régressions elastic-net dont un autre cas fameux est la régression *ridge*.
Contrairement à la régression linéaire classique, elles fonctionnent également
dans un cadre où $p>N$, c’est-à-dire où le nombre de régresseurs est très grand puisque supérieur
au nombre d’observations.

## 1.2 Pénalisation

En adoptant le principe d’une fonction objectif pénalisée,
le LASSO permet de fixer un certain nombre de coefficients à 0.
Les variables dont la norme est non nulle passent ainsi le test de sélection.


<div class="callout callout-style-default callout-tip callout-titled">
<div class="callout-header d-flex align-content-center">
<div class="callout-icon-container">
<i class="callout-icon"></i>
</div>
<div class="callout-title-container flex-fill">
Le programme d’optimisation du LASSO
</div>
</div>
<div class="callout-body-container callout-body">

Le LASSO est un programme d’optimisation sous contrainte. On cherche à trouver l’estimateur $\beta$ qui minimise l’erreur quadratique (régression linéaire) sous une contrainte additionnelle régularisant les paramètres:
$$
\min_{\beta} \frac{1}{2}\mathbb{E}\bigg( \big( X\beta - y  \big)^2 \bigg) \\ 
\text{s.t. } \sum_{j=1}^p |\beta_j| \leq t
$$

Ce programme se reformule grâce au Lagrangien et permet ainsi d’obtenir un programme de minimisation plus maniable :

$$
\beta^{\text{LASSO}} = \arg \min_{\beta} \frac{1}{2}\mathbb{E}\bigg( \big( X\beta - y  \big)^2 \bigg) + \alpha \sum_{j=1}^p |\beta_j| = \arg \min_{\beta} ||y-X\beta||_{2}^{2} + \lambda ||\beta||_1
$$

où $\lambda$ est une réécriture de la régularisation précédente qui dépend de $\alpha$. La force de la pénalité appliquée aux modèles non parcimonieux dépend de ce paramètre.

</div>
</div>

## 1.3 Première régression LASSO

Comme nous cherchons à trouver les
meilleurs prédicteurs du vote Républicain,
nous allons retirer les variables
qui sont dérivables directement de celles-ci : les scores des concurrents !


In [ ]:
import pandas as pd

df2 = pd.DataFrame(votes.drop(columns='geometry'))
df2 = df2.loc[
  :,
  ~df2.columns.str.endswith(
    ('_democrat','_green','_other', 'winner', 'per_point_diff', 'per_dem')
    )
  ]


df2 = df2.loc[:,~df2.columns.duplicated()]

Dans le prochain exercice, nous allons utiliser la fonction suivante pour avoir une matrice de corrélation plus esthétique que celle permise par défaut avec `Pandas`.


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px


def plot_corr_heatmap(
    df: pd.DataFrame,
    drop_cols=None,
    column_labels: dict | None = None,
    decimals: int = 2,
    width: int = 600,
    height: int = 600,
    show_xlabels: bool = False
):
    """
    Trace une heatmap de corrélation (triangle inférieur) à partir d'un DataFrame.

    Paramètres
    ----------
    df : pd.DataFrame
        DataFrame d'entrée.
    drop_cols : list ou None
        Liste de colonnes à supprimer avant le calcul de la corrélation
        (ex: ['winner']).
    column_labels : dict ou None
        Dictionnaire pour renommer les colonnes (ex: column_labels).
    decimals : int
        Nombre de décimales pour l'arrondi avant corr().
    width, height : int
        Dimensions de la figure Plotly.
    show_xlabels : bool
        Afficher ou non les labels en abscisse.
    """
    data = df.copy()

    # 1. Colonnes à drop
    if drop_cols is not None:
        data = data.drop(columns=drop_cols)

    # 2. Arrondi + renommage éventuel
    if column_labels is not None:
        data = data.rename(columns=column_labels)
    data = data.round(decimals)

    # 3. Matrice de corrélation
    corr = data.corr()

    # 4. Masque triangle supérieur
    mask = np.triu(np.ones_like(corr, dtype=bool))
    corr_masked = corr.mask(mask)

    # 5. Heatmap Plotly
    fig = px.imshow(
        corr_masked.values,
        x=corr.columns,
        y=corr.columns,
        color_continuous_scale='RdBu_r',  # échelle inversée
        zmin=-1,
        zmax=1,
        text_auto=".2f"
    )

    # 6. Hover custom
    fig.update_traces(
        hovertemplate="Var 1: %{y}<br>Var 2: %{x}<br>Corr: %{z:.2f}<extra></extra>"
    )

    # 7. Layout
    fig.update_layout(
        coloraxis_showscale=False,
        xaxis=dict(
            showticklabels=show_xlabels,
            title=None,
            ticks=''
        ),
        yaxis=dict(
            showticklabels=show_xlabels,
            title=None,
            ticks=''
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(t=10, b=10, l=10, r=10),
        width=width,
        height=height
    )

    return fig

Dans cet exercice, nous utiliserons
également une fonction pour extraire
les variables sélectionnées par le LASSO,
la voici :


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

def extract_features_selected(lasso: Pipeline, preprocessing_step_name: str = 'preprocess') -> pd.Series:
    """
    Extracts selected features based on the coefficients obtained from Lasso regression.

    Parameters:
    - lasso (Pipeline): The scikit-learn pipeline containing a trained Lasso regression model.
    - preprocessing_step_name (str): The name of the preprocessing step in the pipeline. Default is 'preprocess'.

    Returns:
    - pd.Series: A Pandas Series containing selected features with non-zero coefficients.
    """
    # Check if lasso object is provided
    if not isinstance(lasso, Pipeline):
        raise ValueError("The provided lasso object is not a scikit-learn pipeline.")

    # Extract the final transformer from the pipeline
    lasso_model = lasso[-1]

    # Check if lasso_model is a Lasso regression model
    if not isinstance(lasso_model, Lasso):
        raise ValueError("The final step of the pipeline is not a Lasso regression model.")

    # Check if lasso model has 'coef_' attribute
    if not hasattr(lasso_model, 'coef_'):
        raise ValueError("The provided Lasso regression model does not have 'coef_' attribute. "
                         "Make sure it is a trained Lasso regression model.")

    # Get feature names from the preprocessing step
    features_preprocessing = lasso[preprocessing_step_name].get_feature_names_out()

    # Extract selected features based on non-zero coefficients
    features_selec = pd.Series(features_preprocessing[np.abs(lasso_model.coef_) > 0])

    return features_selec


<div class="callout callout-style-default callout-tip callout-titled">
<div class="callout-header d-flex align-content-center">
<div class="callout-icon-container">
<i class="callout-icon"></i>
</div>
<div class="callout-title-container flex-fill">
Exercice 1 : Premier LASSO
</div>
</div>
<div class="callout-body-container callout-body">

On cherche toujours à prédire la variable `per_gop`. Avant de faire notre estimation, nous allons créer certains objets intermédiaires qui seront utilisés pour
définir notre *pipeline*:

1.  Dans notre `DataFrame`, remplacer les valeurs infinies par des `NaN`.

2.  Créez un échantillon d’entraînement et un échantillon test.

Maintenant nous pouvons passer au coeur de la définition de notre *pipeline*.
[Cet exemple](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) pourra servir de source
d’inspiration, ainsi que [celui-ci](https://www.kaggle.com/code/bextuychiev/lasso-regression-with-pipelines-tutorial).

1.  Créer en premier lieu les étapes
    de *preprocessing* pour notre modèle.
    Pour cela, il est d’usage de séparer les étapes appliquées aux variables numériques continues des autres variables, dites
    catégorielles.

-   Pour les **variables numériques**, imputer à la moyenne puis effectuer une standardisation ;
-   Pour les **variables catégorielles**, les techniques de régression linéaires impliquent d’utiliser une expansion par *one hot encoding*. Avant de faire ce *one hot encoding*, faire une imputation par valeur la plus fréquente.

1.  Finaliser le *pipeline* en ajoutant l’étape d’estimation puis estimer un modèle LASSO pénalisé avec $\alpha = 0.1$.

En supposant que votre *pipeline* soit dans un objet nommé `pipeline` et que la dernière étape
est nommée `model`, vous pouvez
directement accéder à cette étape en utilisant l’objet `pipeline['model']`.

1.  Afficher les valeurs des coefficients. Quelles variables ont une valeur non nulle ?
2.  Montrer que les variables sélectionnées sont parfois très corrélées.
3.  Comparer la performance de ce modèle parcimonieux avec celle d’un modèle avec plus de variables.

<details>

<summary>

Aide pour la question 1

</summary>

``` python
# Remplacer les infinis par des NaN
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
```

</details>

<details>

<summary>

Aide pour la question 3

</summary>

La définition d’un pipeline
suit la structure suivante :

``` python
numeric_pipeline = Pipeline(steps=[
    ('impute', # définir la méthode d'imputation ici
     ),
    ('scale', # définir la méthode de standardisation ici
    )
])

categorical_pipeline = # adapter le template

# À vous de définir en amont numerical_features et categorical_features
preprocessor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, numerical_features),
    ('category', categorical_pipeline, categorical_features)
])
```

</details>

</div>
</div>

Le *pipeline* de *preprocessing* (question 3) prend la forme suivante :


Le *pipeline* prend la forme suivante, une fois finalisé (question 4) :


À l’issue de la question 5, les variables sélectionnées sont :

Le modèle est assez parcimonieux puisqu’un sous-échantillon de nos
variables initiales (d’autant que nos variables catégorielles
ont été éclatées en de nombreuses variables
par le *one hot encoding*).


Certaines variables font sens, comme les variables d’éducation par exemple. Notamment, un des meilleurs prédicteurs pour le score des Républicains en 2020 est… le score des Républicains (et mécaniquement des démocrates) en 2016 et 2012.

Par ailleurs, on sélectionne des variables redondantes. Une phase plus approfondie de nettoyage des données serait en réalité nécessaire.


Le modèle parcimonieux est (légèrement) plus performant :


D’ailleurs, on pourrait déjà remarquer
que régresser le score de 2020 sur celui
de 2016 amène déjà à de très bonnes
performances explicatives, ce qui suggère
que le vote se comporte comme un processus
autorégressif :


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
smf.ols("per_gop ~ share_2016_republican", data = df2).fit().summary()

Notes: [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

# 2. Rôle de la pénalisation $\alpha$ sur la sélection de variables

Nous avons jusqu’à présent pris l’hyperparamètre $\alpha$
comme donné. Quel rôle joue-t-il dans les conclusions de
notre modélisation ? Pour cela, nous pouvons
explorer l’effet que sa valeur peut avoir sur le nombre
de variables passant l’étape de sélection.

Pour le prochain
exercice, nous allons considérer exclusivement
les variables quantitatives pour accélérer les calculs.
En effet, avec des modèles non parcimonieux, les multiples
modalités de nos variables catégorielles rendent le problème
d’optimisation difficilement tractable.


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

df2.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(
    df2.drop(["per_gop"], axis = 1),
    100*df2['per_gop'], test_size=0.2, random_state=0
)

numerical_features = X_train.select_dtypes(include='number').columns.tolist()
categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()

numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

preprocessed_features = pd.DataFrame(
  numeric_pipeline.fit_transform(
    X_train.drop(columns = categorical_features)
  )
)

numeric_pipeline 


<div class="callout callout-style-default callout-tip callout-titled">
<div class="callout-header d-flex align-content-center">
<div class="callout-icon-container">
<i class="callout-icon"></i>
</div>
<div class="callout-title-container flex-fill">
Exercice 2 : Rôle du paramètre de pénalisation
</div>
</div>
<div class="callout-body-container callout-body">

Utiliser la fonction `lasso_path` pour évaluer le nombre de paramètres sélectionnés par LASSO lorsque $\alpha$
varie (parcourir $\alpha \in [0.001,0.01,0.02,0.025,0.05,0.1,0.25,0.5,0.8,1.0]$).

</div>
</div>

La relation que vous devriez obtenir entre $\alpha$ et
le nombre de paramètres est celle-ci :


On voit que plus $\alpha$ est élevé, moins le modèle sélectionne de variables.

# 3. Validation croisée pour sélectionner le modèle

Quel $\alpha$ faut-il privilégier ? Pour cela,
il convient d’effectuer une validation croisée afin de choisir le modèle pour
lequel les variables qui passent la phase de sélection permettent de mieux
prédire le résultat Républicain.


In [ ]:
from sklearn.linear_model import LassoCV

my_alphas = np.array([0.001,0.01,0.02,0.025,0.05,0.1,0.25,0.5,0.8,1.0])

lcv = (
  LassoCV(
    alphas=my_alphas,
    fit_intercept=False,
    random_state=0,
    cv=5
    ).fit(
      preprocessed_features, y_train
    )
)

On peut récupérer le *“meilleur”* $\alpha$ :


In [ ]:
print("alpha optimal :", lcv.alpha_)

alpha optimal : 1.0

Celui-ci peut être utilisé pour faire tourner un nouveau *pipeline* :


In [ ]:
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, numerical_features),
    ('category', categorical_pipeline, categorical_features)
])

model = Lasso(
  fit_intercept=False, 
  alpha = lcv.alpha_
)  

lasso_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', model)
])

lasso_optimal = lasso_pipeline.fit(X_train,y_train)

features_selec2 = extract_features_selected(lasso_optimal)

Les variables sélectionnées sont :

Selected features are:


Cela correspond à un modèle avec 15 variables sélectionnées.


<div class="callout callout-style-default callout-tip callout-titled">
<div class="callout-header d-flex align-content-center">
<div class="callout-icon-container">
<i class="callout-icon"></i>
</div>
<div class="callout-title-container flex-fill">
Tip
</div>
</div>
<div class="callout-body-container callout-body">

Dans le cas où le modèle paraîtrait trop peu parcimonieux, il faudrait revoir la phase de définition des variables pertinentes pour comprendre si des échelles différentes de certaines variables ne seraient pas plus appropriées (par exemple du `log`).

</div>
</div>
